## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join(".." , "Weather_Database" , "WeatherPy_Database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Srandakan,ID,-7.9386,110.2506,75.72,62,100,1.03,overcast clouds
1,1,Palizada,MX,18.2500,-92.0833,69.15,94,5,3.98,clear sky
2,2,Miranda,CO,3.2499,-76.2281,71.73,96,100,1.36,overcast clouds
3,3,Verkhoyansk,RU,67.5447,133.3850,-16.55,90,78,1.16,broken clouds
4,4,Bose,IT,45.4612,8.0137,42.24,51,12,4.27,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 96


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Srandakan,ID,-7.9386,110.2506,75.72,62,100,1.03,overcast clouds
7,7,Bathsheba,BB,13.2167,-59.5167,77.61,53,20,21.85,few clouds
9,9,Cockburn Town,TC,21.4612,-71.1419,77.43,84,3,22.93,clear sky
10,10,La Palma,US,33.8464,-118.0467,79.20,46,0,5.75,clear sky
13,13,Butaritari,KI,3.0707,172.7902,82.36,77,90,16.80,light rain
15,15,Hilo,US,19.7297,-155.0900,76.73,71,100,5.75,overcast clouds
17,17,Bengkulu,ID,-3.8004,102.2655,83.44,62,100,4.03,overcast clouds
18,18,Mahebourg,MU,-20.4081,57.7000,78.26,94,40,5.75,light intensity shower rain
20,20,Saint-Philippe,RE,-21.3585,55.7679,78.39,69,64,5.68,broken clouds
21,21,Manaus,BR,-3.1019,-60.0250,77.49,100,40,3.44,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Srandakan,ID,75.72,overcast clouds,-7.9386,110.2506,
7,Bathsheba,BB,77.61,few clouds,13.2167,-59.5167,
9,Cockburn Town,TC,77.43,clear sky,21.4612,-71.1419,
10,La Palma,US,79.20,clear sky,33.8464,-118.0467,
13,Butaritari,KI,82.36,light rain,3.0707,172.7902,
15,Hilo,US,76.73,overcast clouds,19.7297,-155.0900,
17,Bengkulu,ID,83.44,overcast clouds,-3.8004,102.2655,
18,Mahebourg,MU,78.26,light intensity shower rain,-20.4081,57.7000,
20,Saint-Philippe,RE,78.39,broken clouds,-21.3585,55.7679,
21,Manaus,BR,77.49,scattered clouds,-3.1019,-60.0250,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace('', np.nan)
clean_hotel_df = hotel_df.dropna()

In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))